In [19]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)

In [74]:
df = pd.read_csv('data/aviation_cleaned.csv', index_col=0)
df.head()

,event_id,investigation_type,event_date,location,country,injury_severity,aircraft_damage,aircraft_category,make,model,amateur_built,number_of_engines,engine_type,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status
0,20001218X45444,Accident,1948-10-24,"MOOSE CREEK, ID",United States,Fatal(2),Destroyed,Unknown,Stinson,108-3,No,1,Reciprocating,Personal,Unknown,2,0,0,0,UNK,Cruise,Probable Cause
1,20001218X45447,Accident,1962-07-19,"BRIDGEPORT, CA",United States,Fatal(4),Destroyed,Unknown,Piper,Pa24-180,No,1,Reciprocating,Personal,Unknown,4,0,0,0,UNK,Unknown,Probable Cause
2,20061025X01555,Accident,1974-08-30,"Saltville, VA",United States,Fatal(3),Destroyed,Unknown,Cessna,172M,No,1,Reciprocating,Personal,Unknown,3,0,0,0,IMC,Cruise,Probable Cause
3,20001218X45448,Accident,1977-06-19,"EUREKA, CA",United States,Fatal(2),Destroyed,Unknown,Rockwell,112,No,1,Reciprocating,Personal,Unknown,2,0,0,0,IMC,Cruise,Probable Cause
4,20041105X01764,Accident,1979-08-02,"Canton, OH",United States,Fatal(1),Destroyed,Unknown,Cessna,501,No,0,Unknown,Personal,Unknown,1,2,0,0,VMC,Approach,Probable Cause


In [47]:
# turn all makes and models to titlecase
df['make'] = df['make'].str.strip().str.title()
df['model'] = df['model'].str.strip().str.title()

In [52]:
df.drop(['accident_number', 'airport_code', 'airport_name'], axis=1,inplace=True)

In [71]:
df['aircraft_category'].isin(['Unknown', 'Airplane'])

0        True
1        True
2        True
3        True
4        True
         ... 
88788    True
88789    True
88790    True
88791    True
88792    True
Name: aircraft_category, Length: 88793, dtype: bool

# Replacing all nan values in non-numerical cols with "Unknown"

In [59]:
for col in df.columns:
    df[col] = df[col].fillna('Unknown')

In [61]:
# final aviation_cleand for now
df.to_csv('data/aviation_cleaned.csv')

In [65]:
# create unique make_model column for each specific plane
df['make_model'] = df['make'] + ' ' + df['model']
sorted(list(df[['make']].value_counts().index))

[('107.5 Flying Corporation',),
 ('1200',),
 ('177Mf Llc',),
 ('1977 Colfer-Chan',),
 ('1St Ftr Gp',),
 ('2000 Mccoy',),
 ('2001 Mcgirl',),
 ('2003 Nash',),
 ('2007 Savage Air Llc',),
 ('2021Fx3 Llc',),
 ('3Xtrim',),
 ('5 Rivers Llc',),
 ('67 Flying Dutchman',),
 ('737',),
 ('777',),
 ('781569 Inc',),
 ('85 Manista',),
 ('A Pair Of Jacks',),
 ('A. H. Gettings',),
 ('A. Le Francois',),
 ('A. Schleicher Gmbh & Co.',),
 ('Aaa Aircraft Leasing',),
 ('Aaa Aircraft Llc',),
 ('Aardema Robert John',),
 ('Ab Sportine Aviacija',),
 ('Abbett Gerry',),
 ('Abbey Victor',),
 ('Abbott',),
 ('Abc',),
 ('Abernathy',),
 ('Abraham J G Ii/Erickson K',),
 ('Abrams',),
 ('Abrams/Nuding',),
 ('Abruzzo',),
 ("Ac Mfg & Dev'T Co",),
 ('Aces High',),
 ('Ackert',),
 ('Ackland Jeffrey Dean',),
 ('Acro',),
 ('Acro Sport',),
 ('Acroduster',),
 ('Adams',),
 ('Adams Balloon',),
 ('Adams Balloons Llc',),
 ('Adams Dennis Allen',),
 ('Adams Donald L',),
 ('Adams John R Jr',),
 ('Adams William C',),
 ('Adkins',),
 ('Adler

In [25]:
# 18240 unique planes (some could be duplicates)
df['make_model'].nunique()

18240

In [26]:
# make num_flights that counts number of occurences of each specific plane from original df
make_model_counts = df['make_model'].value_counts().reset_index()
make_model_counts.columns = ['make_model', 'num_flights']
make_model_counts

,make_model,num_flights
0,Cessna 152,2366
1,Cessna 172,1753
2,Cessna 172N,1163
3,Piper Pa-28-140,931
4,Cessna 150,829
...,...,...
18235,Silfvast Starduster Too,1
18236,Cessna C-425,1
18237,Tecnam P92 Echo Super,1
18238,Corvus Carone Mk 1,1


In [31]:
# calculate average injuries for each unique plane
cols = ['total_fatal_injuries', 'total_serious_injuries', 'total_minor_injuries', 'total_uninjured', 'number_of_engines']
models_df = df.groupby('make_model')[cols].mean().reset_index()

# rename to 'avg'
models_df.columns = ['make_model', 'avg_fatal_injuries', 'avg_serious_injuries', 'avg_minor_injuries', 'avg_uninjured', 'avg_num_engines']
models_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18240 entries, 0 to 18239
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   make_model            18240 non-null  object 
 1   avg_fatal_injuries    18240 non-null  float64
 2   avg_serious_injuries  18240 non-null  float64
 3   avg_minor_injuries    18240 non-null  float64
 4   avg_uninjured         18240 non-null  float64
 5   avg_num_engines       16940 non-null  float64
dtypes: float64(5), object(1)
memory usage: 855.1+ KB


In [32]:
# add num_flights to models_df
models_df = models_df.merge(make_model_counts, on='make_model', how='left')
models_df

,make_model,avg_fatal_injuries,avg_serious_injuries,avg_minor_injuries,avg_uninjured,avg_num_engines,num_flights
0,107.5 Flying Corporation One Design Dr 107,1.0,0.0,0.0,0.0,1.0,1
1,1200 G103,0.0,1.0,0.0,0.0,0.0,1
2,177Mf Llc Pitts Model 12,0.0,2.0,0.0,0.0,1.0,1
3,1977 Colfer-Chan Steen Skybolt,0.0,0.0,1.0,0.0,NaN,1
4,1St Ftr Gp Focke-Wulf 190,1.0,0.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...
18235,Zubair S Khan Raven,1.0,0.0,0.0,0.0,1.0,1
18236,Zuber Thomas P Zuber Super Drifter,0.0,0.0,0.0,1.0,1.0,1
18237,Zukowski Eaa Biplane,0.0,0.0,0.0,1.0,1.0,1
18238,Zwart Kit Fox Vixen,0.0,0.0,0.0,2.0,1.0,1


In [13]:
models_df

,make_model,avg_fatal_injuries,avg_serious_injuries,avg_minor_injuries,avg_uninjured,num_flights
0,107.5 Flying Corporation One Design Dr 107,1.0,0.0,0.0,0.0,1
1,1200 G103,0.0,1.0,0.0,0.0,1
2,177Mf Llc Pitts Model 12,0.0,2.0,0.0,0.0,1
3,1977 Colfer-Chan Steen Skybolt,0.0,0.0,1.0,0.0,1
4,1St Ftr Gp Focke-Wulf 190,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...
18235,Zubair S Khan Raven,1.0,0.0,0.0,0.0,1
18236,Zuber Thomas P Zuber Super Drifter,0.0,0.0,0.0,1.0,1
18237,Zukowski Eaa Biplane,0.0,0.0,0.0,1.0,1
18238,Zwart Kit Fox Vixen,0.0,0.0,0.0,2.0,1


In [35]:
models_df[models_df['num_flights'] >= 100].sort_values(by='avg_fatal_injuries').head(20)

,make_model,avg_fatal_injuries,avg_serious_injuries,avg_minor_injuries,avg_uninjured,avg_num_engines,num_flights
8698,Grumman G-164A,0.049315,0.084932,0.145205,0.747945,1.000000,365
3499,Boeing A75N1,0.065421,0.252336,0.177570,1.299065,1.000000,107
9400,Hiller Uh-12E,0.075000,0.206250,0.250000,0.750000,1.000000,160
5319,Cessna T188C,0.075472,0.103774,0.122642,0.707547,0.990099,106
8701,Grumman G-164B,0.078125,0.072917,0.109375,0.786458,1.000000,192
701,Air Tractor At-301,0.098039,0.091503,0.202614,0.614379,1.000000,153
4795,Cessna 188,0.098214,0.116071,0.116071,0.660714,1.000000,112
13338,Piper Pa-20,0.100000,0.053846,0.130769,1.500000,1.000000,130
4568,Cessna 140,0.114713,0.144638,0.182045,1.029925,1.002506,401
4567,Cessna 120,0.119497,0.138365,0.176101,1.006289,1.000000,159


In [141]:
models_df[['avg_fatal_injuries', 'avg_serious_injuries', 'avg_minor_injuries', 'avg_uninjured']].sum(axis=1)

0        1.0
1        1.0
2        2.0
3        1.0
4        1.0
        ... 
18235    1.0
18236    1.0
18237    1.0
18238    2.0
18239    2.0
Length: 18240, dtype: float64

In [144]:
models_df['rough_pass_count'] = models_df[['avg_fatal_injuries', 'avg_serious_injuries', 'avg_minor_injuries', 'avg_uninjured']].sum(axis=1).astype(int)